In [9]:
from mymodule.DatabaseConnection import CRUD
from mymodule.Pipeline import ColumnSelector, DataFrameConverter, SequenceBuilder, TransformerRegressor
from Transformer import PositionalEncoding, TransformerEncoder, Input, Dense, GlobalAveragePooling1D, Dropout, Model

In [1]:
import importlib
import mymodule.DatabaseConnection  # first import
importlib.reload(mymodule) 

<module 'mymodule' from 'c:\\Projects\\Market\\mymodule\\__init__.py'>

In [ ]:
from mymodule.DatabaseConnection import CRUD
from dotenv import load_dotenv
load_dotenv()

In [3]:
lo_stockdailyupdate = CRUD('stockdailyupdate',schema=None)
df = lo_stockdailyupdate.Read(where="Symbol='VEDL'")

In [4]:

# ------------------------------
# ✅ Pipeline Components

# Step 1: Select only 3 columns
column_selector = ColumnSelector(columns=['open', 'high', 'low', 'close', 'volume'])

In [5]:
# Step 2: Define column-wise transformers
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ("num", MinMaxScaler(), ['open', 'high', 'low', 'close', 'volume']),
])

In [6]:
from sklearn.pipeline import Pipeline

features = ['open','high','low','close','volume']

pipeline = Pipeline([
    ("select", ColumnSelector(columns=features)),
    ("scale", MinMaxScaler()),
    ("to_df", DataFrameConverter(columns=features)),
    ("sequence", SequenceBuilder(lookback=60, target_col="close")),
])

In [10]:
def build_transformer_model(sequence_length, feature_dim):

    embed_dim = 64
    num_heads = 4
    ff_dim = 128

    inputs = Input(shape=(sequence_length, feature_dim))

    # Projection to embedding dimension
    x = Dense(embed_dim)(inputs)

    # Positional Encoding
    x = PositionalEncoding(sequence_length, embed_dim)(x)

    # Transformer blocks
    x = TransformerEncoder(embed_dim, num_heads, ff_dim)(x)
    x = TransformerEncoder(embed_dim, num_heads, ff_dim)(x)

    # Pooling instead of Flatten (better for time-series)
    x = GlobalAveragePooling1D()(x)

    # Dense head (same style as your LSTM)
    x = Dropout(0.2)(x)
    x = Dense(32, activation="relu")(x)
    outputs = Dense(1)(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer="adam",
        loss="mean_squared_error"
    )

    return model

In [11]:
X_seq, y_seq = pipeline.fit_transform(df)

model = TransformerRegressor(build_fn=build_transformer_model)
model.fit(X_seq, y_seq)


Epoch 1/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - loss: 0.0580 - val_loss: 0.0043
Epoch 2/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 0.0045 - val_loss: 0.0099
Epoch 3/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 0.0022 - val_loss: 0.0122
Epoch 4/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.0013 - val_loss: 0.0097
Epoch 5/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 0.0011 - val_loss: 0.0138
Epoch 6/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 7.0578e-04 - val_loss: 0.0159
Epoch 7/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - loss: 5.3729e-04 - val_loss: 0.0154
Epoch 8/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 4.2981e-04 - val_loss: 0.0190
Epoch 9/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - loss: 3.9867e-04 - val_loss: 0.0186
Epoch 10/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - loss: 3.5248e-04 - val_loss: 0.0170
Epoch 11/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - loss: 3.7057e-04 - val_loss: 0.0162


,build_fn,<function bui...00215CD1DD760>
,epochs,50
,batch_size,32


In [12]:
class StockPredictor:
    def __init__(self, pipeline, model):
        self.pipeline = pipeline
        self.model = model
        
    def fit(self, df):
        X, y = self.pipeline.fit_transform(df)
        self.model.fit(X, y)
        
    def predict(self, df):
        X, _ = self.pipeline.transform(df)
        return self.model.predict(X[-1:])

In [13]:
predictor = StockPredictor(pipeline, model)
predictor.fit(df)

Epoch 1/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - loss: 0.0810 - val_loss: 0.0082
Epoch 2/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - loss: 0.0050 - val_loss: 0.0040
Epoch 3/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 0.0022 - val_loss: 0.0064
Epoch 4/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 0.0016 - val_loss: 0.0043
Epoch 5/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 6/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - loss: 9.0039e-04 - val_loss: 8.7390e-04
Epoch 7/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - loss: 7.4474e-04 - val_loss: 0.0017
Epoch 8/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 6.7679e-04 - val_loss: 0.0016
Epoch 9/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 6.0699e-04 - val_loss: 0.0025
Epoch 10/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 5.2551e-04 - val_loss: 8.1449e-04
Epoch 11/50
174/174 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 4.8811e-04 - val_loss: 0.0021
Epoch 12

In [14]:
price = predictor.predict(df)
print(price)

c:\Projects\Market\venv\Lib\site-packages\sklearn\pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
[[0.9286546]]


In [15]:
import joblib
joblib.dump(predictor, "./models/vedl_pipeline.pkl")

['./models/vedl_pipeline.pkl']

In [17]:
predictor1 = joblib.load("./models/vedl_pipeline.pkl")
price = predictor1.predict(df)

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {}, 'registered_name': 'Functional', 'build_config': {'input_shape': None}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name': None}, 'loss': 'mean_squared_error', 'loss_weights': None, 'metrics': None, 'weighted_metrics': Non